In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import importlib
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from flearn.utils.model_utils import read_data
import os
import importlib
import random
import numpy as np

from flearn.models import client
from flearn.models import group
from flearn.trainers import grouprox
from flearn.trainers import fedprox
from flearn.trainers import fedavg

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Instructions for updating:
non-resource variables are not supported in the long term
Num GPUs Available:  1


In [2]:
params = {}
#DATASETS = ['sent140', 'nist', 'shakespeare', 'mnist', 'synthetic_iid', 'synthetic_0_0', 'synthetic_0.5_0.5', 'synthetic_1_1']

"""
Parameters that need to be manually set
"""

params['dataset'] = 'sent140'
#params['dataset'] = 'mnist'
#params['model'] = 'mlp'
params['model'] = 'stacked_lstm'
params['optimizer'] = 'grouprox'
params['drop_percent'] = 0

if params['dataset'].startswith('mnist'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.03
  params['mu'] = 1
  params['num_group'] = 3
if params['dataset'] == 'nist':
  params['model_params'] = (26,)
  params['learning_rate'] = 0.003
  params['mu'] = 1
  params['num_group'] = 5
if params['dataset'] == 'sent140':
  params['model_params'] = (25, 2, 100) # seq_len, num_classes, num_hidden
  params['learning_rate'] = 0.3
  params['mu'] = 0.01
  params['num_group'] = 5
if params['dataset'].startswith('synthetic'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.01
  params['mu'] = 1
  params['num_group'] = 5

if params['optimizer']  == 'fedavg':
  pass
if params['optimizer']  == 'fedprox':
  pass
if params['optimizer']  == 'grouprox':
  params['proximal'] = False
  params['RAC'] = False
  params['RCC'] = False
  params['allow_empty'] = True
  
  # TEST Hyperparameters, Experimental Only
  params['evenly'] = False
  params['min_clients'] = 2
    
  # We implement IFCA and FeSEM base on grouprox
  params['ifca'] = False
  params['fesem'] = False
    
"""
Common parameters
"""
params['num_rounds'] = 800
params['num_epochs'] = 20
params['batch_size'] = 10
params['eval_every'] = 1
params['clients_per_round'] = 20 ################ Important ######################
params['seed'] = 233
params['agg_lr'] = 0.05

"""
We immplement IFCA and FeSEM base on grouprox,
Some configurations should be flush for above two CFL frameworks
"""
if params['ifca'] == True or params['fesem'] == True:
    params['proximal'] = False
    params['RCC'] = False
    params['allow_empty'] = True
    params['agg_lr'] = 0

""" Set the output CSV file name """
info = '{}-{}-{}'.format(params['optimizer'], params['dataset'], params['model'])
info2 = ''
suffix = '.csv'
if params['optimizer'] == 'fedavg':
  pass
if params['optimizer'] == 'fedprox':
  pass
if params['optimizer'] == 'grouprox':
    
  if params['ifca'] == True:
    info2 = info2 + '-IFCA'
  if params['fesem'] == True:
    info2 = info2 + '-FeSEM'
  
  info2 = '-NumG{:1d}-Agglr{:.4}-Proximal{:.1}'.format(
    params['num_group'], params['agg_lr'], str(params['proximal']))
  
  if params['RCC'] == True:
    info2 = info2 + '-RCC.{:.1}'.format(str(params['RCC']))
  
  if params['RAC'] == True:
    info2 = info2 + '-RAC.{:.1}-Evenly{:.1}'.format(str(params['RAC']), str(params['evenly']))
  elif params['allow_empty'] == True:
    info2 = info2 + '-Empty{:.1}'.format(str(params['allow_empty']))
  elif params['allow_empty'] == False:
    info2 = info2 + '-Evenly{:.1}'.format(str(params['evenly']))

params['export_filename'] = info + info2 + suffix

print(params['export_filename'])

grouprox-sent140-stacked_lstm-NumG5-Agglr0.05-ProximalF-EmptyT.csv


In [3]:
"""
Some preparations
"""

# read data
train_path = os.path.join('data', params['dataset'], 'data', 'train')
test_path = os.path.join('data', params['dataset'], 'data', 'test')
dataset = read_data(train_path, test_path)

# Load model
if params['dataset'].startswith('synthetic'):  # all synthetic datasets use the same model
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'synthetic', params['model'])
elif params['dataset'].startswith('mnist'):
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'mnist', params['model'])
else:
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', params['dataset'], params['model'])
mod = importlib.import_module(model_path)
learner = getattr(mod, 'Model')

# Set seeds
params['sklearn_seed'] = params['seed']
random.seed(1 + params['seed'])
np.random.seed(12 + params['seed'])
tf.set_random_seed(123 + params['seed'])

# Set server
if params['optimizer'] == 'fedavg':
  server = fedavg.Server(params, learner, dataset)  
if params['optimizer'] == 'fedprox':
  server = fedprox.Server(params, learner, dataset)
if params['optimizer'] == 'grouprox':
  server = grouprox.Server(params, learner, dataset)

Using Group prox to Train
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/home/lab/anaconda3/envs/FL/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:702: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be "
/home/lab/anaconda3/envs/FL/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
/home/lab/anaconda3/envs/FL/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/ho

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

772 Clients in Total
243861 Parameters in Total


Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


Pre-training takes 15.850563526153564s seconds
SVD takes 0.27573370933532715s seconds
Delta w shape: (100, 243861) Matrix V shape: (243861, 3)
Ternary Cossim Matrix calculation takes 0.06590104103088379s seconds
Clustering takes 0.025810718536376953s seconds
Clustering Results: Counter({1: 37, 2: 23, 3: 17, 0: 16, 4: 7})
Clustering Inertia: 2.6587696075439453


In [4]:
"""
Test Ternary Similarity
"""
#x, y = server.test_ternary_cosine_similariy()

'\nTest Ternary Similarity\n'

In [5]:
"""
import matplotlib.pyplot as plt
plt.figure(figsize=(7,7))
plt.scatter(x[:50000], y[:50000], linewidths=0.01, alpha=0.2, norm=True)
plt.show()
"""

'\nimport matplotlib.pyplot as plt\nplt.figure(figsize=(7,7))\nplt.scatter(x[:50000], y[:50000], linewidths=0.01, alpha=0.2, norm=True)\nplt.show()\n'

In [6]:
server.train()

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Training with 20 workers ---


Group 0, clients ['schaeferj89', 'weeps']
Group 1, clients ['AllTheSausages', 'TinaS71', 'coriluvnthedon', 'Mrs_NickJ07', 'mneylon', 'davidj505', 'MacSheikh']
Group 2, clients ['m0po', 'debbier93', 'maddisondesigns', 'weelissa', 'AvonteNikole', 'janabelle_xo', 'nyc_specialist', 'WooopJess']
Group 3, clients ['Japh']
Group 4, clients ['steph_davies', 'marianaguidil']


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Group 0
At round 0 accuracy: 0.48254149971379506
At round 0 training accuracy: 0.4945416479736803
At round 0 training loss: 0.7180128956618322
Group 1
At round 0 accuracy: 0.8721590909090909
At round 0 training accuracy: 0.8947761891412017
At round 0 training loss: 0.3417517058338955
Group 2
At round 0 accuracy: 0.7276760953965613
At round 0 training accuracy: 0.7706810874013447
At round 0 training loss: 0.55480349253568
Group 3
At round 0 accuracy: 0.7010435850214856
At round 0 training accuracy: 0.7125220458553791
At round 0 training loss: 0.6176600418584707
Group 4
At round 0 accuracy: 0.4539877300613497
At round 0 training accuracy: 0.5185589519650655
At round 0 training loss: 0.6898326000206856
At round 0 mean test accuracy: 0.7042668552569543 mean train accuracy: 0.7290937799931887
The groups difference are: [2.02248482 2.29492954 2.2515336  1.94714392 1.91369171]
Begin group  0 training


...

Begin group  2 training


Epoch:  10%|█         | 2/20 [00:00<00:00, 19.75it/s] 

Begin group  3 training


Epoch:  15%|█▌        | 3/20 [00:00<00:00, 21.08it/s] 

Begin group  4 training


In [7]:
"""
fname = 'sampling_interval.csv'
interval = 5
rounds = 200
with open(fname, 'w') as csv_f:
  csv_f.write(str(1)+'\n')
  for i in range(2, rounds+1):
    if i % interval == 0:
      csv_f.write(str(i))
    elif i == rounds:
      csv_f.write('200')
    csv_f.write('\n')
"""

"\nfname = 'sampling_interval.csv'\ninterval = 5\nrounds = 200\nwith open(fname, 'w') as csv_f:\n  csv_f.write(str(1)+'\n')\n  for i in range(2, rounds+1):\n    if i % interval == 0:\n      csv_f.write(str(i))\n    elif i == rounds:\n      csv_f.write('200')\n    csv_f.write('\n')\n"